In [1]:
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
from model import create_model
from align import AlignDlib
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import sys
from PIL import Image
sys.modules['Image'] = Image
from PIL import Image
import warnings
from scipy import ndimage
warnings.filterwarnings('ignore')

In [3]:
# Model for 128 embeddings from a image
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [4]:
# Model Summary
nn4_small2_pretrained.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [5]:
# Initialize the OpenFace face alignment utility
alignment = AlignDlib('weights/landmarks.dat')

In [6]:
# Funtion to align an image using dlib utility
def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [7]:
def load_image(path):
    img = cv2.imread(path, 1)
    return img[...,::-1]

In [8]:
class IdentityMetadata():
    def __init__(self, base, name, file):
        self.base = base
        self.name = name
        self.file = file

    def __repr__(self):
        return self.image_path()
    
    def subdir_path(self):
        return self.name
    
    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path):
    metadata = []
    for i in sorted(os.listdir(path)):
        for f in sorted(os.listdir(os.path.join(path, i))):
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

metadata = load_metadata('images')

In [9]:
datagen = ImageDataGenerator(rotation_range=15, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1,
                             shear_range=0.15, 
                             zoom_range=0.1,
                             channel_shift_range = 10, 
                             brightness_range=[0.2,1.0],
                             horizontal_flip=True)
import os

folder_name='AugmentedImages/'
save_here = folder_name

if not os.path.exists(save_here):
    os.makedirs(save_here) 

    for i, m in enumerate(metadata): 
        img = load_image(m.image_path())
        datagen.fit(np.expand_dims(img,0))
        directory=save_here+m.subdir_path()
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(directory)
        for x, val in zip(datagen.flow(np.expand_dims(img,0), 
                                       save_to_dir=directory, 
                                       save_prefix='AugImage', 
                                       save_format='jpg'),range(3)):
            pass

In [10]:
metadata1 = load_metadata('AugmentedImages')

In [11]:
# To store image embeddings
embedded = np.zeros((metadata1.shape[0], 128))

# Process image through the neural net to get 128 embeddings per image
for i, m in enumerate(metadata1):    
    print(i, " ",m)
    img = load_image(m.image_path())
    img = align_image(img)
    if img is None:
        print("No face found", m)
        continue
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    # obtain embedding vector for image
    embedded[i] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]

0   AugmentedImages\Aastha\AugImage_0_1192.jpg
1   AugmentedImages\Aastha\AugImage_0_1648.jpg
2   AugmentedImages\Aastha\AugImage_0_199.jpg
3   AugmentedImages\Aastha\AugImage_0_2011.jpg
4   AugmentedImages\Aastha\AugImage_0_2039.jpg
5   AugmentedImages\Aastha\AugImage_0_2141.jpg
6   AugmentedImages\Aastha\AugImage_0_2244.jpg
7   AugmentedImages\Aastha\AugImage_0_2245.jpg
8   AugmentedImages\Aastha\AugImage_0_2955.jpg
9   AugmentedImages\Aastha\AugImage_0_3539.jpg
10   AugmentedImages\Aastha\AugImage_0_3745.jpg
11   AugmentedImages\Aastha\AugImage_0_376.jpg
12   AugmentedImages\Aastha\AugImage_0_4094.jpg
13   AugmentedImages\Aastha\AugImage_0_417.jpg
14   AugmentedImages\Aastha\AugImage_0_4286.jpg
15   AugmentedImages\Aastha\AugImage_0_4515.jpg
16   AugmentedImages\Aastha\AugImage_0_4625.jpg
17   AugmentedImages\Aastha\AugImage_0_4714.jpg
18   AugmentedImages\Aastha\AugImage_0_4838.jpg
19   AugmentedImages\Aastha\AugImage_0_564.jpg
20   AugmentedImages\Aastha\AugImage_0_5864.jpg
21   A

In [12]:
from sklearn.preprocessing import LabelEncoder
targets = np.array([m.name for m in metadata1])
encoder = LabelEncoder()
encoder.fit(targets)

# Numerical encoding of identities
y = encoder.transform(targets)

In [13]:
train_idx = np.arange(metadata1.shape[0]) % 2 != 0
test_idx = np.arange(metadata1.shape[0]) % 2 == 0

# 50 train examples of 10 identities (5 examples each)
X_train = embedded[train_idx]
# 50 test examples of 10 identities (5 examples each)
X_test = embedded[test_idx]

y_train = y[train_idx]
y_test = y[test_idx]

In [14]:
### SUPPORT VECTOR CLASSIFIER
# svc = SVC(kernel="linear", C=0.025) #10 -> 57% accuracy, 30-> 31%, 50->45%
# svc = SVC(gamma=2, C=1, probability=True) #30-> 88%, 50->88%

### Gaussian CLASSIFIER
# svc = GaussianProcessClassifier(1.0 * RBF(1.0)) # 30->90%, 50->89%

### KNN CLASSIFIER
# svc = KNeighborsClassifier(2) #10 images -> 85% accuracy, 30,50 images->88%
# svc = KNeighborsClassifier(5) #10 -> 78% accuracy, 30-> 86%, 50->87%

### DECISION TREE CLASSIFIER
# svc = DecisionTreeClassifier(max_depth=5) # 30,50->58%
# svc = DecisionTreeClassifier(max_depth=10) # 30->68%, 50->79%
# svc = DecisionTreeClassifier(max_depth=20) #30-> 70%, 50->80%
# svc = DecisionTreeClassifier(max_depth=30) # 30->73%, 50->80%
# svc = DecisionTreeClassifier(max_depth=40) # 30->73%, 50->78%

### Random Forest Classifier
# svc = RandomForestClassifier(max_depth=10, n_estimators=50, max_features=1)  #10 -> 80% accuracy aprox, 30,50 -> 88%
# svc = RandomForestClassifier(40)  #10 -> 80% accuracy aprox, 30,50 -> 89%
# svc = RandomForestClassifier(30)  #10 -> 70% accuracy aprox, 30,50 -> 89%
# svc = RandomForestClassifier(20)  #10 -> 65% accuracy aprox, 30,50 -> 87%

### MLP CLASSIFIER
svc = MLPClassifier(alpha=1.0, max_iter=1000) # 30,50->88%

### ADA BOOST CLASSIFIER
# svc = AdaBoostClassifier(n_estimators=100, random_state=0) # 30->35%, 50->28%

### QUADRATIC CLASSIFIER
# svc = QuadraticDiscriminantAnalysis() #30->40%, 50->9%

#Naive bayes algorithm
# svc = GaussianNB() #10 -> 73% accuracy aprox, 30 -> 86%, 50 -> 88%

svc.fit(X_train, y_train)

acc_svc = accuracy_score(y_test, svc.predict(X_test))

print(f'SVM accuracy = {acc_svc}')

SVM accuracy = 1.0


In [15]:
# train_idx = np.arange(metadata.shape[0]) 

# # 50 train examples of 10 identities (5 examples each)
# X_train = embedded[train_idx]

# y_train = y[train_idx]

# # svc = KNeighborsClassifier()
# # svc = SVC()
# svc = RandomForestClassifier(20)
# svc.fit(X_train, y_train)


In [16]:
# Prediction Code for live Images

In [17]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predClass = svc.predict([features])
    predClass = encoder.inverse_transform(predClass)[0]
    
    # Display the image with bounding box & label
    cv2.putText(origImg,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
    cv2.rectangle(origImg,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
    cv2.imshow('image',origImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [18]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame1(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predScores = svc.predict_proba([features])
    print(predScores)
    predClass = None
    
    maxProb = (predScores[0]).max()
    
    print(maxProb)
    
    if(maxProb >= 0.8):
    
        predClass = [(predScores[0]).argmax()]
        print(predClass)
        predClass = encoder.inverse_transform(predClass)[0]
    
    
    print(predClass)
    
    return bb, predClass

In [27]:
students = []

cap = cv2.VideoCapture(1)
counter = 0
studentList = []
while(True):
    counter+=1
    # Capture frame-by-frame
    ret, frame = cap.read()
#     print(ret, " ", frame)
    if frame is not None:
        if(alignment.getLargestFaceBoundingBox(frame)!=None):
            bb, predClass = processFrame1(frame)
            print("oo: ",predClass)
            studentList.append(predClass)
            cv2.rectangle(frame,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
            cv2.putText(frame,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
            if counter==10:
                predClass = str(max(set(studentList), key = studentList.count)) 
                if predClass != 'None':
                    if predClass not in students:
                          students.append(predClass)
                    cv2.putText(frame,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
                counter = 0
                studentList = []
#             if(predClass!=None):
#                 cv2.putText(frame,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
#                 if predClass not in students:
#                     students.append(predClass)
        cv2.imshow('frame',frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
#             cv2.destroyAllWindows()
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
print(students)

[[0.13821081 0.10923478 0.75255441]]
0.7525544124109148
None
oo:  None
[[0.02497135 0.95525694 0.01977171]]
0.955256937154787
[1]
AbhishekRanjan
oo:  AbhishekRanjan
[[0.08275816 0.85840309 0.05883875]]
0.8584030934493622
[1]
AbhishekRanjan
oo:  AbhishekRanjan
[[0.06273101 0.84000365 0.09726534]]
0.8400036505890776
[1]
AbhishekRanjan
oo:  AbhishekRanjan
[[0.07055673 0.8377591  0.09168417]]
0.8377591041862278
[1]
AbhishekRanjan
oo:  AbhishekRanjan
[[0.04988792 0.87763542 0.07247666]]
0.8776354217381117
[1]
AbhishekRanjan
oo:  AbhishekRanjan
[[0.05848559 0.88533091 0.0561835 ]]
0.8853309099220136
[1]
AbhishekRanjan
oo:  AbhishekRanjan
[[0.33602987 0.24064162 0.42332851]]
0.42332851001996574
None
oo:  None
[[0.20466657 0.05520005 0.74013338]]
0.7401333803904142
None
oo:  None
[[0.17640024 0.05556823 0.76803153]]
0.7680315301252206
None
oo:  None
[[0.46526957 0.06720762 0.46752281]]
0.4675228123551263
None
oo:  None
[[0.44196129 0.06584589 0.49219282]]
0.49219282307698553
None
oo:  None
[[0